In [22]:
#!/usr/bin/env python
import sys
import math
import linecache

# INSTRUCTIONS !
# The provided code calculates phi coefficients for each code line.
# Make sure that you understand how this works, then modify the provided code
# to work also on function calls (you can use your code from problem set 5 here)
# Use the provide 'mystery' function and test cases for this exercise.
# Remember that for functions the phi values have to be calculated as
# described in the problem set 5 video - 
# you should get 3 phi values for each function - one for positive values (1),
# one for 0 values and one for negative values (-1), called "bins" in the video.
#
# Then combine both approaches to find out the function call and its return
# value that is the most correlated with failure, and then - the line in the
# function. Calculate phi values for the function and the line and put them
# in the variables below. 
# IMPORTANT: The phi value for the line must be the normalized 
# value (i.e. devided by the function phi value)
# Do NOT set these values dynamically.

answer_function = "f2"          # One of f1, f2, f3
answer_bin = -1                 # One of 1, 0, -1
answer_function_phi = 0.6547    # precision to 4 decimal places.
answer_line_phi = 1.0000        # Normalized phi. Precision to 4 decimal places.
# Note: if there are several lines with the same phi value, put them in a list,
# no leading whitespace is required
answer_line = ["elif other < 1:", "grade -= 1"]  # lines of code


# global variable to keep the coverage data in
coverage = {}

# global variable to record the returned values of each function
returned = {}


def traceit(frame, event, arg):
    """Tracing function that tracks line coverage data and the
    returned values of function calls.
    
    The returned values are categorized in three categories as follows:
    -1: If the returned value is less than zero (for numbers), None, NaN or an Exception
     0: If the returned value is zero (for numbers), False or 
        len(returned_value) == 0 (for objects that have a __len__ method)
     1: If the returned value is greater than zero (for numbers), True or 
        len(returned_value) > 0 (for objects that have a __len__ method)
        
    If the returned value is not included in the above, a TypeError is raised."""
    
    global coverage
    global returned

    if event == "line":
        filename = frame.f_code.co_filename
        lineno   = frame.f_lineno
        # This check is only necessary when running this code in Jupyter notebook
        if "Anaconda" in filename:
            return traceit
        if filename not in coverage:
            coverage[filename] = {}
        coverage[filename][lineno] = True
        
    elif event == "return":
        function_name = frame.f_code.co_name
        return_value = arg
        
        # This check is only necessary when running this code in Jupyter notebook
        if function_name not in ["mystery", "f1", "f2", "f3", "test"]:
            return traceit
        
        if not returned.has_key(function_name):
            returned[function_name] = {-1: False, 0: False, 1: False}
        
        if isinstance(return_value, (int, long)):
            if return_value < 0:
                returned[function_name][-1] = True
            elif return_value == 0:
                returned[function_name][0] = True
            else:
                returned[function_name][1] = True
       
        elif isinstance(return_value, float):
            if math.isnan(return_value):
                returned[function_name][-1] = True
            elif return_value < 0:
                returned[function_name][-1] = True
            elif return_value == 0:
                returned[function_name][0] = True
            else:
                returned[function_name][1] = True
                
        elif hasattr(return_value, "__len__"):
            if len(return_value) == 0:
                returned[function_name][0] = True
            else:
                returned[function_name][1] = True
                
        elif return_value is True:
            returned[function_name][1] = True
            
        elif return_value is False:
            returned[function_name][0] = True
            
        elif return_value is None:
            returned[function_name][-1] = True
            
        elif isinstance(return_value, BaseException):
            returned[function_name][-1] = True
            
        else:
            raise TypeError, "Function {} returned an unknown object type".format(function_name)
        
    return traceit

            
def phi(n11, n10, n01, n00):
    """Calculate phi coefficient from given values."""
    return ((n11 * n00 - n10 * n01) / 
             math.sqrt((n10 + n11) * (n01 + n00) * (n10 + n00) * (n01 + n11)))


def print_tables(line_tables, function_tables):
    """Print out values of phi, and result of runs for each covered line
    and for each covered function outcome."""
    
    for filename in line_tables.keys():
        for i in range(243, 300):     # lines of the 'mystery' function in this file
            if line_tables[filename].has_key(i):
                (n11, n10, n01, n00) = line_tables[filename][i]
                try:
                    factor = phi(n11, n10, n01, n00)
                    prefix = "%+.4f%2d%2d%2d%2d" % (factor, n11, n10, n01, n00)
                except:
                    prefix = "       %2d%2d%2d%2d" % (n11, n10, n01, n00)
                    
            else:
                prefix = "               "
                    
            print prefix, linecache.getline(filename, i),
    print
    for function_name in function_tables.keys():
        for i in [-1, 0, 1]:
            (n11, n10, n01, n00) = function_tables[function_name][i]
            try:
                factor = phi(n11, n10, n01, n00)
                prefix = "%+.4f%2d%2d%2d%2d" % (factor, n11, n10, n01, n00)
            except:
                prefix = "       %2d%2d%2d%2d" % (n11, n10, n01, n00)           
                  
            print prefix, "function name = {}".format(function_name), "result = {}".format(i)

            
def run_tests(inputs):
    """Run the 'mystery' function for each test case.
    
    Record and return the tuples:
    (input, outcome, coverage of lines) and (input, outcome, returned value)"""
    global coverage
    global returned
    runs   = []
    
    for test_case in inputs:
        coverage = {}
        returned = {}
        sys.settrace(traceit)
        outcome = mystery(test_case)
        sys.settrace(None) 
        runs.append((test_case, outcome, coverage, returned))
    
    return runs


def init_tables(runs):
    """Create empty tuples for each covered line
    and for each function outcome."""
    
    line_tables = {}
    function_tables ={}
    
    for (test_case, outcome, coverage, returned) in runs:
        
        for filename, lines in coverage.items():
            for line in lines.keys():
                if filename not in line_tables:
                    line_tables[filename] = {}
                if line not in line_tables[filename]:
                    line_tables[filename][line] = (0, 0, 0, 0)
        
        for function_name, results in returned.items():
            if function_name not in function_tables:
                    function_tables[function_name] = {}
            for result, value in results.items():
                function_tables[function_name][result] = (0, 0, 0, 0)
    
    return (line_tables, function_tables)


def compute_n(runs, line_tables, function_tables):
    """Compute n11, n10, etc. for each line and function outcome."""
    
    for filename, lines in line_tables.items():
        for line in lines.keys():
            (n11, n10, n01, n00) = line_tables[filename][line]
            for (test_case, outcome, coverage, returned) in runs:
                if (filename in coverage) and (line in coverage[filename]):
                    # Covered in this run
                    if outcome == "FAIL":
                        n11 += 1  # covered and fails
                    else:
                        n10 += 1  # covered and passes
                else:
                    # Not covered in this run
                    if outcome == "FAIL":
                        n01 += 1  # uncovered and fails
                    else:
                        n00 += 1  # uncovered and passes
            line_tables[filename][line] = (n11, n10, n01, n00)
            
    for function_name, results in function_tables.items():
        for result in results.keys():
            (n11, n10, n01, n00) = function_tables[function_name][result] 
            for (test_case, outcome, coverage, returned) in runs:       
                # Covered in this run
                if (function_name in returned) and returned[function_name][result]:
                    if outcome == "FAIL":
                        n11 += 1  # covered and fails
                    else:
                        n10 += 1  # covered and passes
                # Not covered in this run
                else:
                    if outcome == "FAIL":
                        n01 += 1  # uncovered and fails
                    else:
                        n00 += 1  # uncovered and passes
            
            function_tables[function_name][result] = (n11, n10, n01, n00)
    
    return line_tables, function_tables
      

def mystery(magic):
    assert type(magic) == str
    assert len(magic) > 0
    
    r1 = f1(magic)
    
    r2 = f2(magic)
    
    r3 = f3(magic)
    
    print magic, r1, r2, r3

    if r1 < 0 or r3 < 0:
        return "FAIL"
    elif (r1 + r2 + r3) < 0:
        return "FAIL"
    elif r1 == 0 and r2 == 0:
        return "FAIL"
    else:
        return "PASS"

def f1(ml):
    if len(ml) <6:
        return -1
    elif len(ml) > 12 :
        return 1
    else:
        return 0
    
def f2(ms):
    digits = 0
    letters = 0
    for c in ms:
        if c in "1234567890":
            digits += 1
        elif c.isalpha():
            letters += 1
    other = len(ms) - digits - letters
    grade = 0
    
    if (other + digits) > 3:
        grade += 1 
    elif other < 1:
        grade -= 1
           
    return grade

def f3(mn):
    forbidden = ["pass", "123", "qwe", "111"]
    grade = 0
    for word in forbidden:
        if mn.find(word) > -1:
            grade -= 1
    if mn.find("%") > -1:
        grade += 1
    return grade


# These are the input values you should test the mystery function with
inputs = ["aaaaa223%", "aaaaaaaatt41@#", "asdfgh123!", "007001007",
          "143zxc@#$ab", "3214&*#&!(", "qqq1dfjsns", "12345%@afafsaf"]

runs = run_tests(inputs)
line_tables, function_tables = init_tables(runs)
line_tables, function_tables  = compute_n(runs, line_tables, function_tables)

print_tables(line_tables, function_tables)      

aaaaa223% 0 1 1
aaaaaaaatt41@# 1 1 0
asdfgh123! 0 1 -1
007001007 0 1 0
143zxc@#$ab 0 1 0
3214&*#&!( 0 1 0
qqq1dfjsns 0 -1 0
12345%@afafsaf 1 1 0
                
                def mystery(magic):
        2 6 0 0     assert type(magic) == str
        2 6 0 0     assert len(magic) > 0
                    
        2 6 0 0     r1 = f1(magic)
                    
        2 6 0 0     r2 = f2(magic)
                    
        2 6 0 0     r3 = f3(magic)
                    
        2 6 0 0     print magic, r1, r2, r3
                
        2 6 0 0     if r1 < 0 or r3 < 0:
+0.6547 1 0 1 6         return "FAIL"
-0.6547 1 6 1 0     elif (r1 + r2 + r3) < 0:
+0.6547 1 0 1 6         return "FAIL"
-1.0000 0 6 2 0     elif r1 == 0 and r2 == 0:
                        return "FAIL"
                    else:
-1.0000 0 6 2 0         return "PASS"
                
                def f1(ml):
        2 6 0 0     if len(ml) <6:
                        return -1
        2 6 0 0     elif len(ml) > 12 :
